In [10]:
import mido 
import numpy as np
import os
from music21 import *

In [11]:
#@title Instruments { run: "auto" }

Piano = False #@param {type:"boolean"}
Drum = True #@param {type:"boolean"}
Guitar = True #@param {type:"boolean"}
Bass = True #@param {type:"boolean"}
String = True #@param {type:"boolean"}
Violin = False #@param {type:"boolean"}
Saxophone = False #@param {type:"boolean"}

Piano_NoteInterval = 0.4 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
Drum_NoteInterval = 0.5 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
Guitar_NoteInterval = 0.7 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
Bass_NoteInterval = 0.5 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
String_NoteInterval = 0.6 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
Violin_NoteInterval = 0.5 #@param {type:"slider", min:0.2, max:1.2, step:0.1}
Saxophone_NoteInterval = 0.5 #@param {type:"slider", min:0.2, max:1.2, step:0.1}

### Preprocessing Block

In [40]:
filename = 'data/HighQualityJazz/composition(4).mid'
mdf = mido.MidiFile(filename)

tttime = 0
for t in mdf.tracks:
    for m in t:
        tttime += m.time

tpb = mdf.ticks_per_beat
num_bars = tttime / (tpb * 4)
print('total duration =', tttime, 'ticks per beat =', tpb, 'number of bars =', num_bars)

note_array = []
for t in mdf.tracks:
    for m in t:
        if not m.is_meta and m.type in ['note_on']:
             note_array += [m.note]

min_note = sorted(note_array)[0]
max_note = sorted(note_array)[-1]

print('lowest note =', min_note)


total duration = 27904 ticks per beat = 1024 number of bars = 6.8125
lowest note = 38


In [36]:
from music21 import *

def generate_base_drum(midiTrack, total_time, tpb):
    # setup drum track metamessage
    midiTrack.append(mido.MetaMessage('track_name', name='Base Drums', time=0))
    midiTrack.append(mido.MetaMessage('midi_port', port=0, time=0))
    midiTrack.append(mido.Message('program_change', program=00, channel=9, time=0))
    
    # fill the track
    curr_track_duration = 0
    while curr_track_duration < total_time:
        # generate by bar, 1 bar = 4 beats since in 4/4
        bar_duration = tpb * 4
        oct_beat = np.random.randint(2,4,1)
        for i in range(1, 5):
            beats_ary = []
            if i == oct_beat: # or change to 3/4 for steady effect
                oct_msg1 = mido.Message('note_on', channel=9, note = 41, velocity=64,time=120)
                oct_msg2 = mido.Message('note_on', channel=9, note = 51, velocity=64,time=120)
                oct_msg3 = mido.Message('note_on', channel=9, note = 44, velocity=64,time=240)
                beats_ary = [oct_msg1, oct_msg2, oct_msg3]
            else:
                msg1 = mido.Message('note_on', channel=9, note = 51, velocity=64,time=240)
                msg2 = mido.Message('note_off', channel=9, note = 51, velocity=64,time=240)
                beats_ary = [msg1, msg2]

            # insert beat to track
            for b in beats_ary:
                midiTrack.append(b)
        curr_track_duration += bar_duration

def generate_walking_bass(midiTrack, total_time, tpb, lowest_note):
    lowest_note -= 16
    # setup bass track metamessage
    midiTrack.append(mido.MetaMessage('track_name', name='Bass', time=0))
    midiTrack.append(mido.MetaMessage('midi_port', port=1, time=0))
    midiTrack.append(mido.Message('program_change', channel=0, program=32, time=0))
   
    # fill the track
    curr_track_duration = 0
    current_bar = 0
    while curr_track_duration < total_time:
        # bar variation
        current_bar += 1

        if current_bar % 8 == 3:
            lowest_note += 2
        elif current_bar % 8 == 5:
            lowest_note -= 1
        elif current_bar % 8 == 6:
            lowest_note += 3
               
        # generate by bar, 1 bar = 4 beats since in 4/4
        bar_duration = tpb * 8
        bar_array = []

        # rst_msg = mido.Message('note_off', channel=0, note=lowest_note, velocity=64, time=2-0)
        msg1 = mido.Message('note_on', channel=0, note=lowest_note, velocity=64, time=480)
        msg2 = mido.Message('note_on', channel=0, note=lowest_note+2, velocity=64, time=480)
        msg3 = mido.Message('note_on', channel=0, note=lowest_note+3, velocity=64, time=480)
        msg4 = mido.Message('note_on', channel=0, note=lowest_note+5, velocity=64, time=480)
        msg5 = mido.Message('note_on', channel=0, note=lowest_note-2, velocity=64, time=480)
        rst_beat = mido.Message('note_off', channel=0, note=lowest_note, velocity=64, time=480)

        bar_array = [msg1, msg2, msg3, msg4, msg2, msg5, msg1, rst_beat]

        for b in bar_array:
            midiTrack.append(b)
        
        if current_bar % 8 == 3:
            lowest_note -= 2
        elif current_bar % 8 == 5:
            lowest_note += 1
        elif current_bar % 8 == 6:
            lowest_note -= 3

        curr_track_duration += bar_duration

NOTES_FLAT = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
NOTES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def NoteToMidi(KeyOctave):
    # KeyOctave is formatted like 'C#3'
    key = KeyOctave[:-1]  # eg C, Db
    octave = KeyOctave[-1]   # eg 3, 4
    if not octave in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']:
        KeyOctave += '1'
        return NoteToMidi(KeyOctave)
    answer = -1

    try:
        if 'b' in key:
            pos = NOTES_FLAT.index(key)
        else:
            pos = NOTES_SHARP.index(key)
    except:
        print('The key is not valid', key)
        return answer

    answer += pos + 12 * (int(octave) + 1) + 1
    return answer

def get_chords(midiFD):
    chords = []
    input_midi = converter.parse(midiFD)
    single_track = input_midi[0]    # Known single track file
    voice = single_track.getElementsByClass(stream.Voice)
    full_steam = stream.Voice()
    full_steam.append([j.flat for i,j in enumerate(input_midi)])
    mono_stream = full_steam[0]     # full_stream is flattened so only one stream exist
    # remove other info, only keep chords
    mono_stream.removeByClass(note.Note)
    mono_stream.removeByClass(note.Rest)
    # mono_stream.show('text')
    # full_steam[0].show('text')
    for i, j in enumerate(full_steam[0]):
        if not i == 0:
            chord = []
            for n in j:
                namestr = n.nameWithOctave
                if '-' in list(namestr):
                    namestr = namestr.replace('-', 'b')
                chord += [namestr]
            chords += [chord]
    return chords

def generate_chord_piano(midiTrack, total_time, tpb, origin_track_fd):
    # grab chords
    chords = get_chords(origin_track_fd)
    # setup metamessage
    midiTrack.append(mido.MetaMessage('track_name', name='Piano', time=0))
    midiTrack.append(mido.MetaMessage('midi_port', port=2, time=0))
    midiTrack.append(mido.Message('program_change', channel=0, program=00, time=0))

    # fill the track
    curr_track_duration = 0
    current_bar = 0
    while curr_track_duration < total_time:
        # bar variation
        chord_idx = current_bar % len(chords)    
        next_idx = (current_bar + 1) % len(chords)   
        # generate by bar, 1 bar = 4 beats since in 4/4
        bar_duration = tpb * 11
        bar_array = []

        for note in chords[chord_idx]:
            note_val = NoteToMidi(note)
            chord_msg1 = mido.Message('note_on', channel=0, note=note_val, velocity=64, time=360)
            chord_msg2 = mido.Message('note_on', channel=0, note=note_val-3, velocity=64, time=360)
            chord_msg3 = mido.Message('note_on', channel=0, note=note_val+3, velocity=64, time=360)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3)

            chord_msg1 = mido.Message('note_off', channel=0, note=note_val, velocity=64, time=120)
            chord_msg2 = mido.Message('note_off', channel=0, note=note_val-3, velocity=64, time=120)
            chord_msg3 = mido.Message('note_off', channel=0, note=note_val+3, velocity=64, time=120)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3) 
            
            chord_msg1 = mido.Message('note_on', channel=0, note=note_val, velocity=64, time=180)
            chord_msg2 = mido.Message('note_on', channel=0, note=note_val-3, velocity=64, time=180)
            chord_msg3 = mido.Message('note_on', channel=0, note=note_val+3, velocity=64, time=180)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3)

            chord_msg1 = mido.Message('note_off', channel=0, note=note_val, velocity=64, time=60)
            chord_msg2 = mido.Message('note_off', channel=0, note=note_val-3, velocity=64, time=60)
            chord_msg3 = mido.Message('note_off', channel=0, note=note_val+3, velocity=64, time=60)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3)

        for note in chords[next_idx]:
            note_val = NoteToMidi(note)
            chord_msg1 = mido.Message('note_on', channel=0, note=note_val, velocity=64, time=180)
            chord_msg2 = mido.Message('note_on', channel=0, note=note_val-3, velocity=64, time=180)
            chord_msg3 = mido.Message('note_on', channel=0, note=note_val+3, velocity=64, time=180)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3)

            chord_msg1 = mido.Message('note_off', channel=0, note=note_val, velocity=64, time=60)
            chord_msg2 = mido.Message('note_off', channel=0, note=note_val-3, velocity=64, time=60)
            chord_msg3 = mido.Message('note_off', channel=0, note=note_val+3, velocity=64, time=60)

            midiTrack.append(chord_msg1)
            midiTrack.append(chord_msg2)
            midiTrack.append(chord_msg3) 

        current_bar += 1    
        curr_track_duration += bar_duration

def generate_solo_track(midiFile, original_track):
    replicate_track = mido.MidiTrack()
    replicate_track.append(mido.MetaMessage('track_name', name='Piano Solo', time=0))
    replicate_track.append(mido.MetaMessage('midi_port', port=3, time=0))
    replicate_track.append(mido.Message('program_change', channel=0, program=27, time=0))

    for msg in original_track:
        if not msg.is_meta:
            replicate_track.append(msg)

    midiFile.tracks.append(replicate_track)

In [41]:
debug_midi = mido.MidiFile()

generate_solo_track(debug_midi, mdf.tracks[0])

chord_track = mido.MidiTrack()
generate_chord_piano(chord_track, tttime, tpb, filename)
debug_midi.tracks.append(chord_track)

drum_track = mido.MidiTrack()
generate_base_drum(drum_track, tttime, tpb)
debug_midi.tracks.append(drum_track)

bass_track = mido.MidiTrack()
generate_walking_bass(bass_track, tttime, tpb, min_note)
debug_midi.tracks.append(bass_track)

debug_midi.save('results/Jazzified_song_4.mid')

ValueError: data byte must be in range 0..127